# Exp N2: 2D Phase Diagram (High Noise: η=0.6, 0.8)

## 目的
λ × η の2次元相図を作成し、相転移の全体構造を可視化する。
N2は高ノイズ率（η=0.6, 0.8）を担当。

## 実験設計
- **λ**: 0.20, 0.30, 0.35, 0.40, 0.45, 0.50, 0.60（7点）
- **η（ノイズ率）**: 0.6, 0.8（2点）
- **Seeds**: 0, 1, 2, 3, 4（5個）
- **Total**: 7 × 2 × 5 = **70 runs**

## N1, N2 の関係
- **N1**: η = 0.2, 0.4（低ノイズ）
- **N2（本実験）**: η = 0.6, 0.8（高ノイズ）
- 両方を統合して完全な2D相図を作成

## 推定時間
~70 runs × 9.5 min ≈ **11 時間**

In [ ]:
# ===== セットアップ =====
from google.colab import drive
drive.mount('/content/drive')

import os
from datetime import datetime

EXP_NAME = 'exp_N2_phase_diagram_high_noise'
TIMESTAMP = datetime.now().strftime('%Y%m%d_%H%M%S')
BASE_DIR = '/content/drive/MyDrive/dual-gradient-learning/Paper-A'
SAVE_DIR = f'{BASE_DIR}/{EXP_NAME}_{TIMESTAMP}'
os.makedirs(SAVE_DIR, exist_ok=True)
os.makedirs(f'{SAVE_DIR}/figures', exist_ok=True)

print(f'Experiment: {EXP_NAME}')
print(f'Timestamp: {TIMESTAMP}')
print(f'Save directory: {SAVE_DIR}')

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from torch.nn.utils import parameters_to_vector
import torchvision
import torchvision.transforms as transforms
import numpy as np
import json
import time

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
if torch.cuda.is_available():
    torch.backends.cudnn.benchmark = True
    print(f'GPU: {torch.cuda.get_device_name(0)}')
print(f'Device: {device}')

In [ ]:
# ===== モデル定義 =====
class IndexedDataset(Dataset):
    def __init__(self, dataset):
        self.dataset = dataset
    def __getitem__(self, idx):
        img, label = self.dataset[idx]
        return img, label, idx
    def __len__(self):
        return len(self.dataset)

class BasicBlock(nn.Module):
    expansion = 1
    def __init__(self, in_planes, planes, stride=1):
        super().__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, 3, stride, 1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, 3, 1, 1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, planes, 1, stride, bias=False),
                nn.BatchNorm2d(planes))
    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        return F.relu(out)

class ResNet18(nn.Module):
    def __init__(self, num_classes=10):
        super().__init__()
        self.in_planes = 64
        self.conv1 = nn.Conv2d(3, 64, 3, 1, 1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(64, 2, 1)
        self.layer2 = self._make_layer(128, 2, 2)
        self.layer3 = self._make_layer(256, 2, 2)
        self.layer4 = self._make_layer(512, 2, 2)
        self.linear = nn.Linear(512, num_classes)
    def _make_layer(self, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for s in strides:
            layers.append(BasicBlock(self.in_planes, planes, s))
            self.in_planes = planes
        return nn.Sequential(*layers)
    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.adaptive_avg_pool2d(out, 1)
        out = out.view(out.size(0), -1)
        return self.linear(out)

In [ ]:
# ===== 実験パラメータ =====
BATCH_SIZE = 256
NUM_WORKERS = 4
EPOCHS = 100
LR = 0.1
K = 16

# N2: 高ノイズ率
LAMBDAS = [0.20, 0.30, 0.35, 0.40, 0.45, 0.50, 0.60]
NOISE_RATES = [0.6, 0.8]  # 高ノイズ
SEEDS = [0, 1, 2, 3, 4]

# 実験条件リスト生成
experiments = []
for noise_rate in NOISE_RATES:
    for lam in LAMBDAS:
        for seed in SEEDS:
            experiments.append({
                'lambda': lam,
                'noise_rate': noise_rate,
                'seed': seed
            })

total_runs = len(experiments)
print(f'Lambdas: {LAMBDAS}')
print(f'Noise rates: {NOISE_RATES}')
print(f'Seeds: {SEEDS}')
print(f'Total runs: {total_runs}')
print(f'Estimated time: {total_runs * 9.5 / 60:.1f} hours')

# config保存
config = {
    'experiment': EXP_NAME,
    'timestamp': TIMESTAMP,
    'description': '2D phase diagram - high noise rates (eta=0.6, 0.8)',
    'parameters': {
        'lambdas': LAMBDAS,
        'noise_rates': NOISE_RATES,
        'seeds': SEEDS,
        'epochs': EPOCHS,
        'batch_size': BATCH_SIZE,
        'lr': LR,
        'K': K
    },
    'total_runs': total_runs,
    'series': 'N (2D phase diagram)',
    'part': '2 of 2 (high noise)'
}
with open(f'{SAVE_DIR}/exp_N2_config.json', 'w') as f:
    json.dump(config, f, indent=2)
print(f'Config saved')

In [ ]:
# ===== ユーティリティ関数 =====
def set_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)

def load_cifar10():
    transform_train = transforms.Compose([
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))])
    transform_test = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))])
    trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
    testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)
    return trainset, testset

def get_data_loaders(trainset, testset):
    indexed_trainset = IndexedDataset(trainset)
    train_loader = DataLoader(indexed_trainset, batch_size=BATCH_SIZE, shuffle=True,
                              num_workers=NUM_WORKERS, pin_memory=True, persistent_workers=True, drop_last=True)
    test_loader = DataLoader(testset, batch_size=BATCH_SIZE, shuffle=False,
                             num_workers=NUM_WORKERS, pin_memory=True, persistent_workers=True)
    return train_loader, test_loader

def inject_noise(labels, noise_rate, seed):
    np.random.seed(seed + 1000)
    noisy_labels = labels.copy()
    n_noisy = int(noise_rate * len(labels))
    noisy_indices = np.random.choice(len(labels), n_noisy, replace=False)
    for idx in noisy_indices:
        choices = [i for i in range(10) if i != labels[idx]]
        noisy_labels[idx] = np.random.choice(choices)
    return noisy_labels

def evaluate(model, test_loader):
    model.eval()
    correct = total = 0
    with torch.no_grad():
        for inputs, targets in test_loader:
            inputs, targets = inputs.to(device, non_blocking=True), targets.to(device, non_blocking=True)
            outputs = model(inputs)
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
    return correct / total

In [ ]:
# ===== Dual-Gradient Learning =====
def train_dual_gradient(model, train_loader, test_loader, clean_labels, noisy_labels, lam):
    optimizer = optim.SGD(model.parameters(), lr=LR, momentum=0.9, weight_decay=5e-4)
    scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=[50, 75], gamma=0.1)
    criterion = nn.CrossEntropyLoss()
    
    clean_labels_tensor = torch.tensor(clean_labels, device=device)
    noisy_labels_tensor = torch.tensor(noisy_labels, device=device)
    
    cached_g_value = None
    global_step = 0
    
    best_acc = 0
    cos_history = []
    
    for epoch in range(EPOCHS):
        model.train()
        epoch_cos = []
        
        for inputs, _, indices in train_loader:
            inputs = inputs.to(device, non_blocking=True)
            indices = indices.to(device, non_blocking=True)
            batch_noisy = noisy_labels_tensor[indices]
            batch_clean = clean_labels_tensor[indices]
            
            # Structure gradient
            optimizer.zero_grad()
            outputs = model(inputs)
            loss_struct = criterion(outputs, batch_noisy)
            loss_struct.backward(retain_graph=True)
            g_struct = parameters_to_vector([p.grad for p in model.parameters()]).clone()
            
            # Value gradient
            if global_step % K == 0 or cached_g_value is None:
                optimizer.zero_grad()
                outputs = model(inputs)
                loss_value = criterion(outputs, batch_clean)
                loss_value.backward()
                cached_g_value = parameters_to_vector([p.grad for p in model.parameters()]).clone()
            
            # Normalize and mix
            g_struct_norm = g_struct / (g_struct.norm() + 1e-12)
            g_value_norm = cached_g_value / (cached_g_value.norm() + 1e-12)
            
            cos_sim = (g_struct_norm @ g_value_norm).item()
            epoch_cos.append(cos_sim)
            
            g_mix = (1 - lam) * g_struct_norm + lam * g_value_norm
            
            optimizer.zero_grad()
            idx = 0
            for p in model.parameters():
                numel = p.numel()
                p.grad = g_mix[idx:idx+numel].view(p.shape).clone()
                idx += numel
            optimizer.step()
            global_step += 1
        
        scheduler.step()
        cos_history.append(np.mean(epoch_cos))
        
        if (epoch + 1) % 10 == 0:
            acc = evaluate(model, test_loader)
            best_acc = max(best_acc, acc)
    
    final_acc = evaluate(model, test_loader)
    avg_cos = np.mean(cos_history)
    
    return final_acc, max(best_acc, final_acc), avg_cos

In [ ]:
# ===== データ準備 =====
trainset, testset = load_cifar10()
clean_labels = np.array(trainset.targets)
train_loader, test_loader = get_data_loaders(trainset, testset)

print('Data prepared')

# GPU warmup
warmup_model = ResNet18().to(device)
for _ in range(20):
    _ = warmup_model(torch.randn(BATCH_SIZE, 3, 32, 32, device=device))
del warmup_model
torch.cuda.empty_cache()
print('Warmup complete.')

In [ ]:
# ===== メイン実験ループ =====
results = []
checkpoint_file = f'{SAVE_DIR}/exp_N2_checkpoint.json'
completed = set()

if os.path.exists(checkpoint_file):
    with open(checkpoint_file, 'r') as f:
        results = json.load(f)
    for r in results:
        completed.add((r['lambda'], r['noise_rate'], r['seed']))
    print(f'Checkpoint loaded: {len(completed)} runs completed')

run_counter = 0
exp_start = time.time()

for exp in experiments:
    run_counter += 1
    lam = exp['lambda']
    noise_rate = exp['noise_rate']
    seed = exp['seed']
    
    key = (lam, noise_rate, seed)
    if key in completed:
        continue
    
    print(f'\n[{run_counter}/{total_runs}] λ={lam:.2f} η={noise_rate} seed={seed}')
    t0 = time.time()
    
    set_seed(seed)
    noisy_labels = inject_noise(clean_labels, noise_rate, seed)
    model = ResNet18().to(device)
    
    final_acc, best_acc, avg_cos = train_dual_gradient(
        model, train_loader, test_loader,
        clean_labels, noisy_labels, lam
    )
    elapsed = time.time() - t0
    
    result = {
        'experiment_id': f'N2-{run_counter:03d}',
        'experiment': EXP_NAME,
        'lambda': lam,
        'noise_rate': noise_rate,
        'seed': seed,
        'test_acc': final_acc,
        'test_error': 1 - final_acc,
        'best_acc': best_acc,
        'best_error': 1 - best_acc,
        'avg_cos_struct_value': avg_cos,
        'time_seconds': elapsed
    }
    results.append(result)
    
    with open(checkpoint_file, 'w') as f:
        json.dump(results, f, indent=2)
    
    status = ' ⚠️ COLLAPSE' if (1 - final_acc) > 0.5 else (' ✅ EXCELLENT' if (1 - final_acc) < 0.15 else '')
    print(f'  Error: {1-final_acc:.4f} | cos: {avg_cos:.4f} | Time: {elapsed/60:.1f} min{status}')
    
    completed_count = len(results)
    avg_time = sum([r['time_seconds'] for r in results]) / completed_count
    remaining = total_runs - completed_count
    eta_hours = (remaining * avg_time) / 3600
    print(f'  Progress: {completed_count}/{total_runs} | ETA: {eta_hours:.1f} hours')

print(f'\n{"="*60}')
print(f'EXPERIMENT N2 COMPLETE')
print(f'Total time: {(time.time()-exp_start)/3600:.2f} hours')
print(f'{"="*60}')

In [ ]:
# ===== 結果保存 =====
import pandas as pd

with open(f'{SAVE_DIR}/exp_N2_results.json', 'w') as f:
    json.dump(results, f, indent=2)

df = pd.DataFrame(results)
df.to_csv(f'{SAVE_DIR}/exp_N2_results.csv', index=False)

print(f'Results saved to {SAVE_DIR}/')

In [ ]:
# ===== 可視化: 部分相図 =====
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

df = pd.DataFrame(results)

fig, axes = plt.subplots(1, 2, figsize=(12, 5))

for i, noise_rate in enumerate(NOISE_RATES):
    ax = axes[i]
    df_n = df[df['noise_rate'] == noise_rate]
    
    stats = df_n.groupby('lambda')['test_error'].agg(['mean', 'std'])
    
    ax.errorbar(stats.index, stats['mean'], yerr=stats['std'],
                marker='o', capsize=4, linewidth=2, markersize=8)
    ax.set_xlabel('λ', fontsize=12)
    ax.set_ylabel('Test Error', fontsize=12)
    ax.set_title(f'η = {int(noise_rate*100)}%', fontsize=13, fontweight='bold')
    ax.grid(alpha=0.3)
    ax.set_xlim([0.15, 0.65])

plt.suptitle('Phase Structure (High Noise)', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.savefig(f'{SAVE_DIR}/figures/exp_N2_phase_curves.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# ===== 部分ヒートマップ =====
pivot = df.pivot_table(values='test_error', index='noise_rate', columns='lambda', aggfunc='mean')

fig, ax = plt.subplots(figsize=(10, 4))
im = ax.imshow(pivot.values, cmap='RdYlGn_r', aspect='auto')

ax.set_xticks(range(len(LAMBDAS)))
ax.set_xticklabels([f'{l:.2f}' for l in LAMBDAS])
ax.set_yticks(range(len(NOISE_RATES)))
ax.set_yticklabels([f'{int(n*100)}%' for n in NOISE_RATES])

ax.set_xlabel('λ', fontsize=12)
ax.set_ylabel('Noise Rate (η)', fontsize=12)
ax.set_title('Partial Phase Diagram (N2: High Noise)', fontsize=13, fontweight='bold')

cbar = plt.colorbar(im, ax=ax)
cbar.set_label('Test Error', fontsize=11)

plt.tight_layout()
plt.savefig(f'{SAVE_DIR}/figures/exp_N2_heatmap.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# ===== サマリー =====
print('\n' + '='*70)
print('EXPERIMENT N2: 2D PHASE DIAGRAM (HIGH NOISE) - SUMMARY')
print('='*70)

for noise_rate in NOISE_RATES:
    print(f'\n--- η = {int(noise_rate*100)}% ---')
    df_n = df[df['noise_rate'] == noise_rate]
    stats = df_n.groupby('lambda')['test_error'].agg(['mean', 'std'])
    best_lambda = stats['mean'].idxmin()
    print(f'  Best λ: {best_lambda} (error: {stats["mean"].min():.4f})')

print(f'\n→ N1 と統合して完全な2D相図を作成予定')
print(f'\nSave directory: {SAVE_DIR}')